<a href="https://colab.research.google.com/github/VincentZuo/fin-models/blob/main/%5BFinal%5D_v8_torch_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up and read data

In [ ]:
!pip install transformers==4.17.0
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 8.2 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
     |████████████████████████████████| 6.6 MB 64.4 MB/s 
     |████████████████████████████████| 880 kB 76.1 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5cf46e045bcd52bcc5547bb1afbb0e2a3cb11d61404fa31857db429bdd34cccc
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 354 kB/s 
     |███████

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch
import transformers
print(transformers.__version__)
print(torch.__version__)

4.17.0
1.11.0+cu113


In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
read_df = pd.read_pickle("/content/gdrive/MyDrive/224U_2022/final_project_data.pkl")

In [ ]:
read_df.head()

,title,label,range,emb
0,Joseph Greenberg,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","(0, 106)","[[-0.07557425, -0.011484241, -0.07807306, 0.00..."
1,Pauline Donalda,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","(106, 127)","[[-0.057122264, 0.005889078, -0.030671664, -0...."
2,List of German football transfers summer 2017,"[0, 0, 1, 0, 0]","(127, 132)","[[-0.0032938644, -0.02310217, -0.05397396, -0...."
3,Lester Hudson,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, ...","(132, 176)","[[-0.0091192825, 0.090078354, -0.029187214, -0..."
4,Monique Ganderton,"[1, 0, 0, 0, 0, 1, 0]","(176, 183)","[[-0.090651, -0.09176681, -0.029592248, -0.062..."


In [ ]:
read_df = read_df.sample(frac=1)
read_df.head()

,title,label,range,emb
793,Henry Charbonneau,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]","(16145, 16158)","[[0.05722959, -0.008566863, -0.094458535, 0.00..."
504,Wild West City,"[1, 0]","(10505, 10507)","[[0.10378061, 0.071708806, -0.028881393, 0.004..."
1007,Mikkel Aagaard,"[1, 1, 0, 0, 1, 0, 0, 1, 0]","(20758, 20767)","[[-0.090821765, 0.018140187, 0.030428424, -0.0..."
1820,Frederik Hendrikplantsoen,"[0, 0]","(38642, 38644)","[[0.13479649, 0.087804824, -0.01524276, -0.064..."
500,Jane Somerville,"[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","(9566, 9631)","[[-0.02891623, -0.043679826, 0.0065269014, -0...."


In [ ]:
MAX_SEQ_LENGTH = 128

In [ ]:
from sentence_transformers import SentenceTransformer
#Define the model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
empty_sentence = sentence_model.encode([''])
MAX_SEQ_LENGTH=128
FAKE_INDEX = 0

In [ ]:
def dummy_emb():
  return empty_sentence[0]

def pad_emb(current_emb, seq_len):
  if len(current_emb) > seq_len:
    return current_emb[0:seq_len]
  padding =  np.array([dummy_emb()] * (seq_len - len(current_emb)), dtype = 'double')
  return np.concatenate([current_emb, padding], axis=0)

def pad_label(current_label, seq_len):
  if len(current_label) > seq_len:
    return current_label[0:seq_len]
  padding =  [FAKE_INDEX] * (seq_len - len(current_label))
  # print(current_label, padding)
  return np.concatenate([current_label, padding], axis=0)

In [ ]:
from torch.utils.data import Dataset

class WikiPandasDfDataSet(Dataset):
  def __init__(self, input_df):
    self.pandas_df = input_df

  def __getitem__(self, index):
    seq = np.array(self.pandas_df.iloc[index]["emb"], dtype='double')
    label = np.array(self.pandas_df.iloc[index]["label"], dtype='double')
    return seq, label

  def __len__(self):
      return len(self.pandas_df)

def collate_fn(batch):
    batched_seq = []
    batched_label = []
    for seqs, labels in batch:
      try:
        local_label = pad_label(labels, MAX_SEQ_LENGTH)
        local_emb = pad_emb(seqs, MAX_SEQ_LENGTH)
        batched_seq.append(local_emb)
        batched_label.append(local_label)
      except Exception as e:
        print("collate_fn err:", e)
        continue
    return torch.tensor(batched_seq).float(), torch.unsqueeze(torch.tensor(batched_label).float(), axis=2)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE_TRAIN = 32

dataset_train = WikiPandasDfDataSet(read_df.head(6400))
train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE_TRAIN, collate_fn = collate_fn, shuffle=True, drop_last=True)
dataset_val = WikiPandasDfDataSet(read_df.tail(3200))
val_dataloader = DataLoader(dataset_val, batch_size=BATCH_SIZE_TRAIN, collate_fn = collate_fn, shuffle=False, drop_last=True)

# NN Model code
https://pytorch.org/tutorials/beginner/translation_transformer.html

In [ ]:
import torch.nn as nn
import datetime
import math
from torch.nn import TransformerEncoderLayer

In [ ]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout=0.1, maxlen=MAX_SEQ_LENGTH):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TransformerClassifierModel(nn.Module):
  def __init__(self, emb_dim=384, nhead = 16):
    super(TransformerClassifierModel, self).__init__()
    self.transformer_encoder = TransformerEncoderLayer(d_model=emb_dim,
                                                        nhead=nhead)
    self.positional_encoding = PositionalEncoding(emb_dim)
    self.classifier_layer = nn.Linear(emb_dim, 1)
    self.activation = torch.nn.Sigmoid()

  def forward(self, seq_input):
    enc_emb = self.positional_encoding(seq_input)
    trans_enc = self.transformer_encoder(enc_emb)
    linear_output = self.classifier_layer(trans_enc)
    return self.activation(linear_output)

# Training loop code

In [ ]:
from tqdm import tqdm
from transformers import AdamW
from sklearn.metrics import f1_score,accuracy_score

In [ ]:
def train(train_dataloader, is_early=False, max_batches=500):
    model.train()

    total_loss, total_accuracy = 0, 0
    step = 0
    # iterate over batches
    for batch in tqdm(train_dataloader):
        sequences, labels = batch[0].to(device),batch[1].to(device)

        model.zero_grad()        
        preds = model(sequences)
        loss = CELoss(preds, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()

        # progress update after every 100 batches.
        if step % 50 == 0 and not step == 0:
            
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
            print("loss",loss.item())
            torch.cuda.empty_cache()
            if is_early and step>=max_batches:
                print("early stopping...")
                break
        step+=1
    # compute the training loss of the epoch
    avg_loss = total_loss / step
    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    #total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss

def evaluate(dev_dataloader, is_early=False, max_batches=500):
  
    print("\nEvaluating...")
  
    # deactivate dropout layers
    model.eval()
    total_loss, total_accuracy = 0, 0
    eval_preds = []
    eval_labels = []
    curr_examples = 0

    # iterate over batches
    step = 0
    for batch in tqdm(dev_dataloader):
    
      # Progress update every 10 batches.
      if step % 10 == 0 and not step == 0:
    
        # # Report progress.
        # temp = np.delete(total_preds,0,0)
        # print('  Batch {:>5,}  of  {:>5,} accuracy {}.'.format(step, len(dev_dataloader), accuracy_score(list(temp[:,0]),list(temp[:,1]))))
        
        # print("F1 score {}".format(f1_score(list(temp[:,0]),list(temp[:,1]),average="macro")))
        if is_early and step>=max_batches:
            print("early stopping...")
            break
      # push the batch to gpu
      #batch = [t.to(device) for t in batch]
  

      sequences, labels = batch[0].to(device),batch[1].to(device)
      curr_examples += len(labels)
      # deactivate autograd
      with torch.no_grad():
    
      # model predictions
          preds = model(sequences)

          # compute the validation loss between actual and predicted values
          loss = CELoss(preds,labels)

          total_loss = total_loss + loss.item()
          eval_preds += preds.detach().cpu().numpy().flatten().tolist()
          eval_labels += labels.detach().cpu().numpy().flatten().tolist()
      step+=1
    # compute the validation loss of the epoch
    avg_loss = total_loss / step 

    return avg_loss, eval_preds, eval_labels

In [ ]:
LEARING_RATE = 0.001
NUM_EPOCS = 5

model = TransformerClassifierModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
CELoss = nn.BCELoss()
optimizer = AdamW(model.parameters(),lr = LEARING_RATE)

train_losses = []
valid_losses = []
accuracy_list = []
f1_list = []

for epoch in range(NUM_EPOCS):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, NUM_EPOCS))
    
    #train model
    train_loss = train(train_dataloader, is_early=True)
    
    #evaluate model
    valid_loss, e_pred, e_label = evaluate(val_dataloader, is_early=True)

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    # accuracy_list.append(accuracy)
    # f1_list.append(valid_f1)

    print('Train loss:', train_loss, 'Val loss:', valid_loss)

    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
      labels_bool = [ elem > 0 for elem in e_label]
      preds_bool = [elem > threshold for elem in e_pred]
      accuracy = accuracy_score(labels_bool, preds_bool)
      valid_f1 = f1_score(labels_bool, preds_bool,average="macro")
      print('Threshold:', threshold,'Accuracy:', accuracy, 'F1:', valid_f1)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,



 Epoch 1 / 5


 26%|██▌       | 52/200 [00:06<00:19,  7.54it/s]

  Batch    50  of    200.
loss 0.21984297037124634


 42%|████▎     | 85/200 [00:10<00:13,  8.31it/s]

...


Evaluating...


 93%|█████████▎| 93/100 [00:10<00:00,  9.02it/s]

collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)


100%|██████████| 100/100 [00:11<00:00,  8.97it/s]


Train loss: 0.09256078088656068 Val loss: 0.08907548669725657
Threshold: 0.1 Accuracy: 0.9047187597686777 F1: 0.7189921386248961
Threshold: 0.2 Accuracy: 0.9047187597686777 F1: 0.7189921386248961
Threshold: 0.3 Accuracy: 0.9066041145670523 F1: 0.7183743339157794
Threshold: 0.4 Accuracy: 0.9541287316348859 F1: 0.4882629870738514
Threshold: 0.5 Accuracy: 0.9541287316348859 F1: 0.4882629870738514
Threshold: 0.6 Accuracy: 0.9541287316348859 F1: 0.4882629870738514


In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)
out = transformer_encoder(src)

In [ ]:
print(out.shape)

torch.Size([10, 32, 512])


In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)

In [ ]:
print(out.shape)

torch.Size([10, 32, 512])


In [ ]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

In [ ]:
print(out.shape)

torch.Size([20, 32, 512])
